In [39]:
import sys
sys.path.append("../transcriber/")
from whisper_call_back import *

In [40]:
model_string = 'ggml-small.en-tdrz.bin'
buffer_size_seconds = 5 
verbose = 0
use_tinydiarize = True
output_file = "transcription.txt"   
# start_recording(buffer_size_seconds=buffer_size_seconds, model_string=model_string, 
#                 verbose=verbose, use_tinydiarize=use_tinydiarize, output_file=output_file)

# Ollama SLM

In [1]:
from ollama_utils import *

In [2]:
# Set the ollama params
ollama_server = 'http://localhost:11434/v1'
model = "llama3"#"mixtral:8x7b"#"llama3"
api_key = "ollama"

In [3]:
# Initialize client
client = init_client(ollama_server,api_key = "ollama")

In [11]:
# text = "hey Carlos, how are you doing? I saw you got a promotion, congrats!"
text = """

Speaker 2  9:32  
I definitely think I could, I will admit you know, a lot of your competitors, they have kind of told us the same thing. And it's kind of been for half the price, I will admit, but I really do like your approach and would be interested in hearing more.

Speaker 1  9:45  
Yeah, I hear you loud and clear. Our competitor may have a lower price, but not nearly what we have to offer. So I'm going to show you another visual and I'm gonna go into that a little bit more. Let me pull it up for you real quick.


"""

In [12]:
# Generate the response
response = run_llm_call(client,model, text)

In [13]:
# Get the string response
string_response = response['choices'][0]['content']
string_response

'{"label":"Rapport_building", "predicted_confidence":0.3},\n{"label":"Needs_assesment", "predicted_confidence":0.2},\n{"label":"Handling_objections", "predicted_confidence":0.4},\n{"label":"Decision_makers", "predicted_confidence":0.05},\n{"label":"Wrapping_up_a_call", "predicted_confidence":0.1},\n{"label":"Other", "predicted_confidence":0.0}'

In [14]:
# Get json response
json_response = extract_json(string_response)

In [15]:
json_response

[{'label': 'Rapport_building', 'predicted_confidence': 0.3},
 {'label': 'Needs_assesment', 'predicted_confidence': 0.2},
 {'label': 'Handling_objections', 'predicted_confidence': 0.4},
 {'label': 'Decision_makers', 'predicted_confidence': 0.05},
 {'label': 'Wrapping_up_a_call', 'predicted_confidence': 0.1},
 {'label': 'Other', 'predicted_confidence': 0.0}]

In [16]:

# Find the dictionary with the maximum predicted_confidence
max_confidence_dict = max(json_response, key=lambda x: x['predicted_confidence'])

# Extract the label and confidence
argmax_label = max_confidence_dict['label']
argmax_confidence = max_confidence_dict['predicted_confidence']

print(f"Label with highest confidence: {argmax_label}")
print(f"Confidence: {argmax_confidence}")

Label with highest confidence: Handling_objections
Confidence: 0.4


In [ ]:
text_strings = [
    "I am wondering what SOCTYPE certification you all have.",
    "I heard you may have some integration difficulty from past vendors."
    "What security compliance certification do you all have?",
    "What's the cost difference between your basic and premium plans?",
    "How does your product compare to Gong in terms of features?",
    "Can your software handle high volumes of concurrent users without slowing down?",
    "Does your platform comply with GDPR regulations for storing user data?",
    "Is there a discount for annual billing versus monthly?",
    "I've heard Outreach just released a similar feature. How is yours different?",
    "We experienced some downtime with our current provider. What's your uptime guarantee?",
    "How do you ensure data encryption at rest and in transit?"
]

In [ ]:
len(text_strings)

10

In [ ]:
import torch
from transformers import pipeline

# Check if CUDA is available
device = 0 if torch.cuda.is_available() else -1


# Initialize the zero-shot classification pipeline
classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli",
                      device = device)

In [ ]:
# Define the categories
categories = ["pricing", "competition", "technical_issues", "security_compliance"]

# Function to classify text
def classify_text(text):
    result = classifier(text, categories)
    return result['labels'][0], result['scores'][0]

In [ ]:
# Classify each text
for text in text_strings:
    category, confidence = classify_text(text)
    print(f"Text: {text}")
    print(f"Category: {category}")
    print(f"Confidence: {confidence:.2f}")
    print()

Text: I am wondering what SOCTYPE certification you all have.
Category: security_compliance
Confidence: 0.36

Text: I heard you may have some integration difficulty from past vendors.What security compliance certification do you all have?
Category: security_compliance
Confidence: 0.60

Text: What's the cost difference between your basic and premium plans?
Category: pricing
Confidence: 0.89

Text: How does your product compare to Gong in terms of features?
Category: competition
Confidence: 0.68

Text: Can your software handle high volumes of concurrent users without slowing down?
Category: technical_issues
Confidence: 0.54

Text: Does your platform comply with GDPR regulations for storing user data?
Category: security_compliance
Confidence: 0.59

Text: Is there a discount for annual billing versus monthly?
Category: pricing
Confidence: 0.87

Text: I've heard Outreach just released a similar feature. How is yours different?
Category: competition
Confidence: 0.53

Text: We experienced som

In [ ]:
text_string

'I heard you may have security issues'

In [ ]:
classify_text(text_string)

('technical_issues', 0.6248490214347839)

In [ ]:

# Define the guideline checklist
guideline_checklist = ["rapport building","assessing needs", "handling an objection", 
                       "identifying decision makers", "wrapping up a call"]

# Function to classify text
def classify_text(text, categories):
    result = classifier(text, categories)
    return result['labels'][0], result['scores'][0]



In [ ]:
text = "hey Carlos, how are you doing? I saw you got a promotion, congrats!"

In [ ]:
classify_text(text, guideline_checklist)

('identifying decision makers', 0.48995521664619446)

In [ ]:
# Generate the response
response = run_llm_call(client,model, text_string)

KeyboardInterrupt: 

In [ ]:
response